# Labels: Descriptive stats

In [1]:
import sys

import pandas as pd
import numpy as np

sys.path.append('../..')
from data import constants

# Read training and test data
df_train = pd.read_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Labels_Training.pkl")
df_test = pd.read_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Labels_Test.pkl")

In [2]:
df_train.groupby(['cro', 'cro_sub_type_combined']).size().to_frame('Train')

Train
cro cro_sub_type_combined       
OP  PRODUCTS                 423
    RESILIENCE               423
PR  ACUTE                    113
    CHRON                     21
TR  MARKET                    51
    POLICY                   109
    REPUTATION                29

In [6]:
len(df_train.query('cro != cro'))

26708

In [9]:
df_train_sum = df_train.groupby(['cro', 'cro_sub_type_combined']).size().to_frame('Training')
df_test_sum = df_test.groupby(['cro', 'cro_sub_type_combined']).size().to_frame('Test')

data = pd.concat([df_train_sum, df_test_sum], axis=1)  # pd.DataFrame.concat([df_train_sum, df_test_sum])
data.rename(index=constants.map_to_label(), inplace=True)
data = data.reindex(constants.cro_category_labels2, level=0, copy=False)
data = data.reindex(constants.cro_sub_category_labels2, level=1, copy=False)
data.index = data.index.rename([' ', ' '])
data.loc['Positive paragraphs', :] = data.sum().values
data.loc['Negative paragraphs', :] = [len(df_train.query('cro != cro')), len(df_test.query('cro != cro'))]
data

Training     Test
                                                                       
Physical risk       Acute                                113.0     21.0
                    Chronic                               21.0      7.0
Transition risk     Policy & Legal                       109.0     20.0
                    Market & Technology                   51.0     12.0
                    Reputation                            29.0      7.0
Opportunity         Products, Services & Markets         423.0     18.0
                    Resource Efficiency & Resilience     423.0      4.0
Positive paragraphs                                     1169.0     89.0
Negative paragraphs                                    26708.0  28132.0

In [10]:


def add_hline(latex: str, index: int) -> str:
    """
    Adds a horizontal `index` lines before the last line of the table

    Args:
        latex: latex table
        index: index of horizontal line insertion (in lines)
    """
    lines = latex.splitlines()
    lines.insert(len(lines) - index - 2, r'\midrule')
    return '\n'.join(lines).replace('NaN', '')

latex_str = data.to_latex(float_format="%.0f")
latex_str = add_hline(latex_str, 2)
print(latex_str)
with open('/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Thesis/tables/cro_label_count.tex', 'w') as tf:
     tf.write(latex_str)

\begin{tabular}{llrr}
\toprule
                    &                                  &  Training &  Test \\
  &   &           &       \\
\midrule
Physical risk & Acute &       113 &    21 \\
                    & Chronic &        21 &     7 \\
Transition risk & Policy \& Legal &       109 &    20 \\
                    & Market \& Technology &        51 &    12 \\
                    & Reputation &        29 &     7 \\
Opportunity & Products, Services \& Markets &       423 &    18 \\
                    & Resource Efficiency \& Resilience &       423 &     4 \\
\midrule
Positive paragraphs &                                  &      1169 &    89 \\
Negative paragraphs &                                  &     26708 & 28132 \\
\bottomrule
\end{tabular}


In [ ]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = df_train.groupby(['cro', 'cro_sub_type_combined']).count()

g = sns.catplot(x="cro_sub_type_combined", col="cro", col_wrap=4,
                data=df_train,
                kind="count", height=2.5, aspect=.8)

In [ ]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Setup seaborn
sns.set_theme(style="ticks", rc={'text.usetex' : True})
sns.set_context("paper")

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
groups.plot(ax=axes[0], kind='bar')
sub_groups.plot(ax=axes[1], kind='bar')